## Setting the environment for Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
%cd "/content/drive/My Drive/Colab Notebooks/w266_final/project_re"

/content/drive/.shortcut-targets-by-id/1I3W7Z7rz_YfsjBjX7z4zTzGbr-iKTrmD/w266_final/project_re


In [3]:
#pip installations for colab
!pip install transformers
!pip install pytorch_pretrained_bert

In [4]:
%load_ext autoreload
#%autoreload
import os, json
from types import SimpleNamespace
from experiment import run_model
from eval import calculate_stats
import pickle
from datetime import datetime 
import torch

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"

In [6]:
config_folder = "config/"

class obj(object):
    def __init__(self, d):
        for a, b in d.items():
            if isinstance(b, (list, tuple)):
               setattr(self, a, [obj(x) if isinstance(x, dict) else x for x in b])
            else:
               setattr(self, a, obj(b) if isinstance(b, dict) else b)

with open(os.path.join(config_folder, "config.json")) as f:
    config = obj(json.load(f))

In [7]:
train_inputs, train_labels, train_preds, train_loss, dev_inputs, dev_labels, dev_loss, dev_preds = run_model(config, device)

INFO:root:***** Running training *****
INFO:root:  Num examples = 61014
INFO:root:  Batch size = 12
INFO:root:  Num steps = 25420
Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
Epoch:  20%|██        | 1/5 [16:50<1:07:22, 1010.65s/it]


so far epoch accuracy:  0.9425541678958927


Epoch:  40%|████      | 2/5 [33:46<50:36, 1012.07s/it]  


so far epoch accuracy:  0.9862490575933393


...

Epoch:  80%|████████  | 4/5 [1:07:23<16:50, 1010.52s/it]


so far epoch accuracy:  0.9925099157570394


Epoch: 100%|██████████| 5/5 [1:24:12<00:00, 1010.44s/it]


so far epoch accuracy:  0.9936735831120727
Training complete
Best val Acc: 0.993674


INFO:root:***** Running training *****
INFO:root:  Num examples = 15304
INFO:root:  Batch size = 12
INFO:root:  Num steps = 6375


In [8]:
train_inputs, train_labels, train_preds, train_loss, dev_inputs, dev_labels, dev_loss, dev_preds

(array([[  101,   152,  2064, ...,     0,     0,     0],
        [  101,  1109,  1701, ...,     0,     0,     0],
        [  101,  9583,  1104, ...,     0,     0,     0],
        ...,
        [  101,   152,  2064, ...,     0,     0,     0],
        [  101, 20338, 15969, ...,     0,     0,     0],
        [  101,   141,   120, ...,     0,     0,     0]]),
 array([1, 8, 8, ..., 5, 8, 3]),
 array([1, 8, 8, ..., 5, 8, 3]),
 0.021908083500653312,
 array([[  101,  1153,  1110, ...,     0,     0,     0],
        [  101,   152,  2064, ...,     0,     0,     0],
        [  101,   108,   156, ...,     0,     0,     0],
        ...,
        [  101,  1153,  1108, ...,     0,     0,     0],
        [  101,   108,   164, ...,     0,     0,     0],
        [  101, 16752, 25352, ...,     0,     0,     0]]),
 array([8, 3, 0, ..., 8, 8, 8]),
 0.10624750547817555,
 array([8, 3, 0, ..., 8, 8, 8]))

In [9]:
import numpy as np
np.array(dev_labels).shape, np.array(dev_preds).shape, np.array(train_preds).shape, np.array(train_labels).shape

((15304,), (15304,), (61014,), (61014,))

In [10]:
%autoreload
train_mcc, train_f1_score, train_df_results, train_label_matches_df = calculate_stats(train_labels,train_preds )
dev_mcc, dev_f1_score, dev_df_results, dev_label_matches_df = calculate_stats(dev_labels,dev_preds )


 label: [1 8 8 ... 5 8 3]

 preds: [1 8 8 ... 5 8 3]

 label: [8 3 0 ... 8 8 8]

 preds: [8 3 0 ... 8 8 8]


In [11]:
all_experiment_results = []
all_experiment_results.append([str(config), train_loss, dev_loss, train_mcc, train_f1_score,dev_mcc,dev_f1_score, 
                               dev_labels, dev_preds,train_labels,train_preds, train_inputs, dev_inputs])
len(all_experiment_results[0])

13

In [12]:
dev_label_matches_df

predicted
labels         matched           
ADE-Drug       False           18
               True           232
Dosage-Drug    False           22
               True           911
Duration-Drug  False            5
               True           131
Form-Drug      False           33
               True           921
Frequency-Drug False           21
               True          1248
Reason-Drug    False           30
               True          1004
Route-Drug     False           30
               True          1115
Strength-Drug  False           32
               True          1350
no relation    False           13
               True          8188

In [13]:
all_model_results_pickle_file = config.programsettings.REPORTS_DIR + "multi_model_experiment_results_" + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
with open(all_model_results_pickle_file, "wb") as f:
    pickle.dump(all_experiment_results, f)

In [15]:
from sklearn.metrics import classification_report
print(classification_report(dev_labels,dev_preds ))

              precision    recall  f1-score   support

           0       0.98      0.97      0.97      1034
           1       0.97      0.97      0.97      1145
           2       0.98      0.98      0.98      1382
           3       0.99      0.98      0.99      1269
           4       0.95      0.96      0.96       136
           5       0.96      0.97      0.96       954
           6       0.97      0.98      0.97       933
           7       0.88      0.93      0.90       250
           8       1.00      1.00      1.00      8201

    accuracy                           0.99     15304
   macro avg       0.96      0.97      0.97     15304
weighted avg       0.99      0.99      0.99     15304



## Analyzing results

In [ ]:
with open("reports/re/multi_model_experiment_results_2020-11-19 20_25_02_351087.pkl", "rb") as f:
  all_model_results_pickle_file = pickle.load(f)

In [ ]:
all_model_results_pickle_file

[['<__main__.obj object at 0x7f098018dac8>',
  0.012548324644060544,
  0.047653015470251976,
  0.9938410610583791,
  0.9921212179856703,
  0.9823099468862545,
  0.9710182287919125,
  array([6, 8, 8, ..., 5, 8, 1]),
  array([6, 8, 8, ..., 5, 8, 1]),
  array([8, 8, 8, ..., 2, 8, 0]),
  array([8, 8, 8, ..., 2, 8, 0])]]

In [ ]:
len(all_model_results_pickle_file[0])

11

In [ ]:
str_config = all_model_results_pickle_file[0][0]
train_loss = all_model_results_pickle_file[0][1]
dev_loss = all_model_results_pickle_file[0][2]
train_mcc = all_model_results_pickle_file[0][3]
train_f1_score = all_model_results_pickle_file[0][4]
dev_mcc = all_model_results_pickle_file[0][5]
dev_f1_score = all_model_results_pickle_file[0][6]
all_dev_label_ids = all_model_results_pickle_file[0][7]
dev_preds = all_model_results_pickle_file[0][8]
train_label_ids = all_model_results_pickle_file[0][9]
train_preds = all_model_results_pickle_file[0][10]

In [ ]:
str_config

'<__main__.obj object at 0x7f098018dac8>'

In [ ]:
%autoreload
train_mcc, train_f1_score, train_df_results, train_label_matches_df = calculate_stats(train_label_ids,train_preds )
dev_mcc, dev_f1_score, dev_df_results, dev_label_matches_df = calculate_stats(all_dev_label_ids,dev_preds )


 label: [8 8 8 ... 2 8 0]

 preds: [8 8 8 ... 2 8 0]

 label: [6 8 8 ... 5 8 1]

 preds: [6 8 8 ... 5 8 1]


In [ ]:
dev_df_results[(dev_df_results['matched'] == False) & (dev_df_results['labels']=='ADE-Drug')]

,labels,predicted,matched
985,ADE-Drug,Reason-Drug,False
986,ADE-Drug,Reason-Drug,False
2184,ADE-Drug,Reason-Drug,False
2554,ADE-Drug,Reason-Drug,False
4251,ADE-Drug,Reason-Drug,False
4299,ADE-Drug,Reason-Drug,False
4783,ADE-Drug,Reason-Drug,False
5196,ADE-Drug,Reason-Drug,False
5380,ADE-Drug,no relation,False
6326,ADE-Drug,Reason-Drug,False


In [ ]:
dev_df[dev_df['relation_code'] == 7].iloc[0:20, :]

,id,relation_code,alpha,string,dev_preds,dev_true_ids
10,56,7,a,Since no new infection was found this was presumed [**12-26**] OBJ_B steroids OBJ_E and the SUB_B leukocytosis SUB_E improved with prednisone taper.,0,0
79,407,7,a,"History of Present Illness: 38 yo F w/ h/o ALL in remission s/p cord transplant in [**1-13**], OBJ_B anthracycline OBJ_E-induced SUB_B cardiomyopathy SUB_E (EF 15-20% [**1-14**]) and recurrent nausea and vomiting who presents with abdominal pain, N/V x1 week Of note, the pt was admitted here from [**Date range (1) **] with nausea and vomitting of unclear etiology.",8,8
109,575,7,a,"- underwent phase I induction with daunorubicin, vincristine, dexamethasone, L-asparaginase, MTX; phase II with cyclophosphamide, cytarabine, mercaptopurine, MTX - Bone Marrow Aspirate/Biopsy on [**2172-10-26**] showed no morphologic evidence of residual leukemia - underwent allo double cord blood SCT [**2173-1-11**], course complicated by neutropenic fever and acute skin GVHD OTHER MEDICAL HISTORY: - Embolic stroke in [**3-/2174**] on coumadin - SUB_B Cardiomyopathy SUB_E due to early OBJ_B anthracycline OBJ_E-related cardiotoxicity [**10/2172**] - Chronic kidney disease stage III/IV, baseline creatinine ~2.0-2.2 - Asthma - HTN - Cervical Intraepithelial neoplasia - C-section in [**2165**]",8,8
110,576,7,a,"- underwent phase I induction with daunorubicin, vincristine, dexamethasone, L-asparaginase, MTX; phase II with cyclophosphamide, cytarabine, mercaptopurine, MTX - Bone Marrow Aspirate/Biopsy on [**2172-10-26**] showed no morphologic evidence of residual leukemia - underwent allo double cord blood SCT [**2173-1-11**], course complicated by neutropenic fever and acute skin GVHD OTHER MEDICAL HISTORY: - Embolic stroke in [**3-/2174**] on coumadin - Cardiomyopathy due to early OBJ_B anthracycline OBJ_E-related SUB_B cardiotoxicity SUB_E [**10/2172**] - Chronic kidney disease stage III/IV, baseline creatinine ~2.0-2.2 - Asthma - HTN - Cervical Intraepithelial neoplasia - C-section in [**2165**]",6,6
217,1143,7,a,Coumadin was thus held and reversed with FFP and vitamin K given her history of SUB_B GIB SUB_E on OBJ_B anticoagulation OBJ_E.,8,8
495,2584,7,a,He became quite SUB_B agitated SUB_E after OBJ_B Naloxone OBJ_E so he was given 2.5mg IV Haldol.,8,8
499,2607,7,a,"# Altered Mental Status: As above, felt to be secondary to SUB_B hypercarbia SUB_E and OBJ_B narcotics OBJ_E.",2,2
558,2904,7,a,"She did not experience significant nausea during hospitalization, but continued to have SUB_B diarrhea SUB_E related to her OBJ_B chemotherapy OBJ_E which was treated with Lomotil.",1,1
612,3192,7,a,"You were seen by dermatology who thought that you had a SUB_B drug sensitivity reaction SUB_E. Your rash appeared to improve after discontinuation of the antibiotics OBJ_B piperacillin OBJ_E and tazobactam, but the exact cause was unknown.",6,6
796,4187,7,a,Status post OBJ_B erlotinib OBJ_E 150 mg/day from [**4-2**] to [**2156-4-22**] (intolerant to medication due to grade [**2-6**] SUB_B rash SUB_E).,8,8


In [ ]:
print(len(dev_preds))
print(len(dev_preds) == len(all_dev_label_ids))

15304
True


In [ ]:
#import numpy as np
#missed_outputs = np.where((dev_preds) != (all_dev_label_ids))[0]

In [ ]:
#len(missed_outputs)

184

In [ ]:
import pandas as pd
dev_df = pd.read_csv("data_divided/dev.tsv", sep = "\t", header=None)
dev_df = dev_df.rename(columns={0: "id", 1: "relation_code", 2: "alpha", 3:"string"})

In [ ]:
len(dev_df)

15304

In [ ]:
dev_df['dev_preds'] = pd.Series(dev_preds)
dev_df['dev_true_ids'] = pd.Series(all_dev_label_ids)

In [ ]:
pd.set_option('display.max_colwidth', -1)
dev_df[(dev_df['dev_preds'] != dev_df['dev_true_ids']) & (dev_df['relation_code']==7)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,id,relation_code,alpha,string,dev_preds,dev_true_ids
2722,13470,7,a,Allergies: OBJ_B Erythromycin OBJ_E causes rash and SUB_B itching SUB_E.,2,1
11891,59272,7,a,"Discharge Disposition: Home Discharge Diagnosis: SUB_B hypotension SUB_E secondary to pneumovax, OBJ_B beta blocker OBJ_E Paroxysmal atrial fibrillation Bipolar disorder History of rheumatic heart disease Status post clipping of cerebral aneurysm in [**2167**] at the [**Hospital1 756**] Gastroesophageal reflux disease.",1,2
13814,69030,7,a,There was then a second episode of SUB_B LFT elevation SUB_E that resolved with stopping OBJ_B zonisamide OBJ_E.,3,2
14291,71375,7,a,She was noted to develop an arm SUB_B rash SUB_E and eyelid edema when she got the levaquin and OBJ_B morphine OBJ_E.,8,0
14394,71810,7,a,"4) Thrombocytopenia: Thought to be [**1-26**] drug effect of voriconazole vs. ranitidine; however, per PCP at [**Name9 (PRE) 2025**], he has had baseline SUB_B thrombocytopenia SUB_E. Thrombocytopenia has been mentioned as a rare side effect of vori, but per ID's assessment of risks vs. benefits, would not stop OBJ_B Voriconazole OBJ_E.",7,0


In [ ]:
dev_df['dev_preds'].value_counts()

8    8201
2    1382
3    1269
1    1145
0    1034
5     954
6     933
7     250
4     136
Name: dev_preds, dtype: int64

In [ ]:
dev_df['dev_true_ids'].value_counts()

8    8203
2    1378
3    1278
1    1142
0    1032
5    951 
6    935 
7    250 
4    135 
Name: dev_true_ids, dtype: int64

##  Try with different configuration

### Just change model from BIOR to BERT Sequence

In [ ]:
config.programsettings.MODEL_NAME = "BERT_Sequence"
config.programsettings.DEBUG_PRINT = 0

In [ ]:
train_label_ids, train_preds, train_loss, all_dev_label_ids, dev_loss, dev_preds = run_model(config, device)

INFO:root:***** Running training *****
INFO:root:  Num examples = 61014
INFO:root:  Batch size = 12
INFO:root:  Num steps = 50840
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.modeling:extracting archive file cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c to temp dir C:\Users\pnarsina\AppData\Local\Temp\2\tmp8umad1wl
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_v

C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epoch:  10%|███████▎                                                                 | 1/10 [12:02<1:48:22, 722.50s/it]


so far epoch accuracy:  0.9336709607631035


Epoch:  20%|██████████████▌                                                          | 2/10 [24:03<1:36:16, 722.08s/it]


so far epoch accuracy:  0.9750876847936539


...

Epoch:  90%|█████████████████████████████████████████████████████████████████▋       | 9/10 [1:48:11<12:01, 721.01s/it]


so far epoch accuracy:  0.986265447274396


Epoch: 100%|████████████████████████████████████████████████████████████████████████| 10/10 [2:00:12<00:00, 721.21s/it]


so far epoch accuracy:  0.9869374241977251
Training complete
Best val Acc: 0.986937


INFO:root:***** Running training *****
INFO:root:  Num examples = 15304
INFO:root:  Batch size = 12
INFO:root:  Num steps = 12750


In [ ]:

train_mcc, train_f1_score, train_df_results, train_label_matches_df = calculate_stats(train_label_ids,train_preds )
dev_mcc, dev_f1_score, dev_df_results, dev_label_matches_df = calculate_stats(all_dev_label_ids,dev_preds)


 label: [5 8 6 ... 1 2 8]

 preds: [5 8 6 ... 1 2 8]

 label: [8 0 8 ... 8 8 8]

 preds: [8 0 8 ... 8 8 8]


In [ ]:
all_experiment_results.append([str(config), train_loss, dev_loss, train_mcc, train_f1_score,dev_mcc,dev_f1_score, 
                               all_dev_label_ids, dev_preds,train_label_ids,train_preds  ])
all_experiment_results

[['<__main__.obj object at 0x00000145C2150790>',
  0.028281437373610104,
  0.10090719395104036,
  0.9828714870380854,
  0.9811421345806985,
  0.9655254255851637,
  0.9513203962222588],
 ['<__main__.obj object at 0x00000145C2150790>',
  0.029586662868419118,
  0.09663268666182578,
  0.9826565895445114,
  0.9813933562820968,
  0.9658135876808309,
  0.9538088513676909,
  array([8, 0, 8, ..., 8, 8, 8], dtype=int64),
  array([5, 8, 6, ..., 1, 2, 8], dtype=int64)]]

In [ ]:
dev_label_matches_df

predicted
labels         matched           
ADE-Drug       False           31
               True           219
Dosage-Drug    False           25
               True           908
Duration-Drug  False            8
               True           128
Form-Drug      False           60
               True           894
Frequency-Drug False           53
               True          1216
Reason-Drug    False           39
               True           995
Route-Drug     False           42
               True          1103
Strength-Drug  False           60
               True          1322
no relation    False           38
               True          8163

In [ ]:

all_model_results_pickle_file = config.programsettings.REPORTS_DIR + "multi_model_experiment_results_" + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
with open(all_model_results_pickle_file, "wb") as f:
    pickle.dump(all_experiment_results, f)  


In [ ]:
for i in range(0,50):
    print (all_dev_label_ids[i], dev_preds[i])


8 8
0 0
8 8
0 0
6 6
1 1
2 2
2 2
0 0
8 8
8 8
8 8
2 2
6 6
0 0
8 8
0 0
6 6
6 6
6 6
2 2
8 8
8 8
2 2
8 8
8 8
6 6
0 0
6 6
8 8
8 8
0 0
8 8
0 8
8 8
5 5
8 8
8 8
8 8
8 8
8 8
8 8
8 8
2 2
8 8
8 8
0 0
8 8
0 0
8 8


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(all_dev_label_ids,dev_preds ))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95      1034
           1       0.97      0.96      0.97      1145
           2       0.94      0.96      0.95      1382
           3       0.94      0.96      0.95      1269
           4       0.97      0.94      0.96       136
           5       0.95      0.94      0.94       954
           6       0.98      0.97      0.97       933
           7       0.91      0.88      0.89       250
           8       1.00      1.00      1.00      8201

    accuracy                           0.98     15304
   macro avg       0.96      0.95      0.95     15304
weighted avg       0.98      0.98      0.98     15304

